# Forecasting 


In this lesson, we will practice forecasting using the following methods:  

- **Last observed value**  
- **Simple average**  
- **Moving average**
- **Previous cycle** 
- **Holt's linear trend**  
- **Holt's seasonal trend**
 

______________________________


We will acquire and prepare store sales data stored in our Codeup MySQL server, then forecast using the modeling approached listed above and evaluate performances.

In [ ]:
# for presentation purposes
import warnings
warnings.filterwarnings("ignore")

# wrangle
from env import user, password, host
import os

# transform
import numpy as np
import pandas as pd

# visualize 
import matplotlib.pyplot as plt
import seaborn as sns

# working with dates
from datetime import datetime

# modeling
import statsmodels.api as sm
from statsmodels.tsa.api import Holt, ExponentialSmoothing
np.random.seed(0)

# evaluate
from sklearn.metrics import mean_squared_error
from math import sqrt 

## Wrangle


We will acquire the data from our MySQL server and run the following prepare steps:
- Convert the `sale_date` column to be a datetime type
- Set the `sale_date` column as our index
- Rename `sale_amount` to `quantity` for clarity
- Create a new field called `sales_total` that is the product of `quantity` and `item_price`

In [ ]:
def get_db_url(database):
    '''
    Returns a formatted string using credentials stored in env.py that can be passed to a pd.read_sql() function
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{database}'
    
def get_store_data():
    '''
    Returns a dataframe of all store data in the tsa_item_demand database and saves a local copy as a csv file.
    '''
    query = '''
            SELECT *
            FROM items
            JOIN sales USING(item_id)
            JOIN stores USING(store_id)
            '''
    
    df = pd.read_sql(query, get_db_url('tsa_item_demand'))
    df.to_csv('tsa_store_data.csv', index=False)
    return df

def wrangle_store_data():
    '''
    Checks for a local cache of tsa_store_data.csv and if not present will run the get_store_data() function which acquires data from Codeup's mysql server
    '''
    filename = 'tsa_store_data.csv'
    if os.path.isfile(filename):
        df = pd.read_csv(filename)
    else:
        df = get_store_data()
    return df

def prep_store_data(df):
    '''
    Prepares raw store data for analysis and time series modeling.
    '''
    df.sale_date = pd.to_datetime(df.sale_date)
    df = df.set_index('sale_date').sort_index()
    df = df.rename(columns={'sale_amount': 'quantity'})
    df['sales_total'] = df.quantity * df.item_price
    return df

In [ ]:
df = wrangle_store_data()
df = prep_store_data(df)
df.head()

Let's take a look at some of the details of our data:

In [ ]:
df.info()

In [ ]:
df.describe()

For the purposes of our modeling, we will only work with one variable at a time (univariate). To this end, we can resample our dataframe to a daily time period, aggregating only the `quantity` and `sales_total` fields:

In [ ]:
df_resampled = df.resample('d')[['quantity','sales_total']].sum()
df_resampled.head()

## Split

We will use a percentage based approach to splitting our time series data.  
1. Identify the total length of the dataframe and multiply by our desired perentage to get the number of rows that equates to the first x% of the dataframe, which equates to the first x% of the time covered in the data.

In [ ]:
df_resampled.shape

In [ ]:
# set train size to be 50% of total 
train_size = int(round(df_resampled.shape[0] * 0.5))
train_size

In [ ]:
# set validate size to be 30% of total 
validate_size = int(round(df_resampled.shape[0] * 0.3))
validate_size

In [ ]:
# set test size to be number of rows remaining. 
test_size = int(round(df_resampled.shape[0] * 0.2))
test_size

In [ ]:
len(df_resampled) == train_size + validate_size + test_size

In [ ]:
# validate will go from 913 to 913+548
validate_end_index = train_size + validate_size
validate_end_index

2. Slice our dataframe using the index positions we identified for each section in the previous step.

In [ ]:
# train will go from 0 to 912
train = df_resampled[:train_size]

In [ ]:
train.tail()

The last observation for train occurred on `2015-07-02`. We will want to make sure that the first observation for validate is the very next day.

In [ ]:
# validate will go from 912 to 1458
validate = df_resampled[train_size:validate_end_index]

In [ ]:
validate.head()

The first observation for validate is `2015-07-03`. This means there is no gap in dates between train and validate. Likewise, we will want to make sure that the last observation in validate is adjacent to the first observation in test.

In [ ]:
validate.tail()

In [ ]:
# test will include 1459 to the end
test = df_resampled[validate_end_index:]

In [ ]:
test.head()

In [ ]:
train.shape[0], validate.shape[0], test.shape[0]

**Verify Splits**

Does the length of each df equate to the length of the original df? 

In [ ]:
# is len of train + validate + test == lenght of entire dataframe. 
len(train) + len(validate) + len(test) == len(df_resampled)

Does the first row of original df equate to the first row of train? 

In [ ]:
print(df_resampled.head(1) == train.head(1))

Is the last row of test the same as the last row of our original dataframe? 

In [ ]:
pd.concat([test.tail(1), df_resampled.tail(1)])

## Visualizing Our Data

Let's plot our data first, viewing where the data is split into train and test. 

In [ ]:
train.columns

In [ ]:
for col in train.columns:
    plt.figure(figsize=(14,8))
    plt.plot(train[col], color='#377eb8', label = 'Train')
    plt.plot(validate[col], color='#ff7f00', label = 'Validate')
    plt.plot(test[col], color='#4daf4a', label = 'Test')
    plt.legend()
    plt.ylabel(col)
    plt.title(col)
    plt.show()

## Creating Helpful Evaluation Functions

Before we try out different methods for forecasting sales and number of items sold, let's create a couple of functions that will be helpful in evaluating each of the modeling approaches we will use. 

`evaluate()` will compute the Mean Squared Error and the Root Mean Squared Error of our predictions compared to the actual values.

In [ ]:
def evaluate(target_var):
    '''
    This function will take the actual values of the target_var from validate, 
    and the predicted values stored in yhat_df, 
    and compute the rmse, rounding to 0 decimal places. 
    it will return the rmse. 
    '''
    rmse = round(sqrt(mean_squared_error(validate[target_var], yhat_df[target_var])), 0)
    return rmse

`plot_and_eval()` will use the evaluate function and also plot train and test values with the predicted values in order to visualize our performance.

In [ ]:
def plot_and_eval(target_var):
    '''
    This function takes in the target var name (string), and returns a plot
    of the values of train for that variable, validate, and the predicted values from yhat_df. 
    it will als lable the rmse. 
    '''
    plt.figure(figsize = (12,4))
    plt.plot(train[target_var], label='Train', linewidth=1, color='#377eb8')
    plt.plot(validate[target_var], label='Validate', linewidth=1, color='#ff7f00')
    plt.plot(yhat_df[target_var], label='yhat', linewidth=2, color='#a65628')
    plt.legend()
    plt.title(target_var)
    rmse = evaluate(target_var)
    print(target_var, '-- RMSE: {:.0f}'.format(rmse))
    plt.show()

We are planning on evaluating a lot of models. Let's create an easy to read dataframe called `eval_df`. 

In [ ]:
# create an empty dataframe
eval_df = pd.DataFrame(columns=['model_type', 'target_var', 'rmse'])
eval_df

Rather than manually appending the results of each model, we can create a function that will do it for us. 

`append_eval_df(model_type)` will append evaluation metrics for each model into our `eval_df` data frame object. 

In [ ]:
# function to store the rmse so that we can compare
def append_eval_df(model_type, target_var):
    '''
    this function takes in as arguments the type of model run, and the name of the target variable. 
    It returns the eval_df with the rmse appended to it for that model and target_var. 
    '''
    rmse = evaluate(target_var)
    d = {'model_type': [model_type], 'target_var': [target_var],
        'rmse': [rmse]}
    d = pd.DataFrame(d)
    return eval_df.append(d, ignore_index = True)

## Forecast 

Forecasting is another word for predicting time series data. As a reminder, we will work with the following approaches:

#### Baseline Models
1. Last Observed Value
2. Simple Average
3. Moving Average

#### Non-Baseline Models
4. Previous Cycle
5. Holt's Linear Trend
6. Holt's Seasonal Trend


## Last Observed Value

The simplest method for forecasting is to predict all future values to be the last observed value.  

### Make Predictions

What was the last observed value for `sales_total` in our training data?

In [ ]:
train['sales_total'][-1:][0]

In [ ]:
# take the last item of sales total and assign to variable
last_sales = train['sales_total'][-1:][0]

What was the last observed value for `quantity` in our training data?

In [ ]:
# take the last item of quantity and assign to variable
last_quantity = train['quantity'][-1:][0]
last_quantity

We can create a dataframe containing our predictions (which will all be the same value with this baseline approach):

In [ ]:
yhat_df = pd.DataFrame(
    {'sales_total': [last_sales],
     'quantity': [last_quantity]},
    index=validate.index)

yhat_df.head()

In [ ]:
yhat_df.describe()

The output of `.describe()` confims that every predicted value is the same.  

### Plot Actual vs. Predicted Values

Let's plot actual and predicted values using our `plot_and_eval()` function

In [ ]:
for col in train.columns:
    plot_and_eval(col)

**Evaluate** 

Evaluate using MSE and RMSE, and add evaluation metrics to `eval_df`. We can use our `append_eval_df` function to accomplish this.

In [ ]:
for col in train.columns:
    eval_df = append_eval_df(model_type = 'last_observed_value', 
                             target_var = col)

eval_df

## Simple Average

Take the simple average of historical values in train and use that value to predict future values.   

This is another good option for an initial baseline. Every predicted period (those in 'test') will be assigned the same value (the average of the entire training set).  

**Make Predictions**

In [ ]:
# compute simple average of sales_total (from train data)
avg_sales = round(train['sales_total'].mean(), 2)
avg_sales

In [ ]:
# compute simple average of quantity (from train data)
avg_quantity = round(train['quantity'].mean(), 2)
avg_quantity

Make a prediction for the value of `sales_total` and `quantity`, stored as the dataframe `yhat_df` for every day of the validate index. We can turn this into a function for ease of continued use.

In [ ]:
def make_baseline_predictions(sales_predictions=None, quantity_predictions=None):
    yhat_df = pd.DataFrame({'sales_total': [sales_predictions],
                           'quantity': [quantity_predictions]},
                          index=validate.index)
    return yhat_df

In [ ]:
yhat_df = make_baseline_predictions(avg_sales, avg_quantity)

In [ ]:
yhat_df.head()

In [ ]:
yhat_df.describe()

### Plot Actual vs. Predicted Values

Similar to our handling of the previous baseline model, we can plot our `yhat_df` values against the actual values in validate. Our `plot_and_eval()` function accomplishes this.

In [ ]:
for col in train.columns:
    plot_and_eval(col)

### Evaluate

Evaluate using MSE and RMSE, and add evaluation metrics to `eval_df`

In [ ]:
for col in train.columns:
    eval_df = append_eval_df(model_type='simple_average', 
                            target_var = col)
eval_df

## Moving Average

In this example, we will use a 30-day moving average to forecast. In other words, the average over the last 30-days will be used as the forecasted value. 

**Make Predictions**

There could be several ways to obtain the value of the last 30 periods in `train`. We will use the `.rolling()` method to accomplish this. 

In [ ]:
period=30
train['sales_total'].rolling(period).mean()

`.rolling()` gives us an array of moving averages. We will only need the last one.

In [ ]:
period=30
train['sales_total'].rolling(period).mean()[-1]

In [ ]:
# Saving the last 30 day moving average for each column
rolling_sales = round(train['sales_total'].rolling(period).mean()[-1], 2)
rolling_quantity = round(train['quantity'].rolling(period).mean()[-1], 2)
print(rolling_sales, rolling_quantity)

We can use our `make_baseline_predictions()` function to create the newest version of the `yhat_df`:

In [ ]:
yhat_df = make_baseline_predictions(rolling_sales, rolling_quantity)
yhat_df.head()

### Plot Actual vs. Predicted Values

Now, let's plot and evaluate the performance of our time series model using **Moving Average**

In [ ]:
for col in train.columns:
    plot_and_eval(col)

### Evaluate

Evaluate using MSE and RMSE, and add evaluation metrics to `eval_df`

In [ ]:
for col in train.columns:
    eval_df = append_eval_df(model_type = '30d_moving_avg', 
                            target_var = col)

eval_df

Let's try out several other values for our period:

In [ ]:
periods = [4, 12, 26, 52, 104]

for p in periods: 
    rolling_sales = round(train['sales_total'].rolling(p).mean()[-1], 2)
    rolling_quantity = round(train['quantity'].rolling(p).mean()[-1], 2)
    yhat_df = make_baseline_predictions(rolling_sales, rolling_quantity)
    model_type = str(p) + '_day_moving_avg'
    for col in train.columns:
        eval_df = append_eval_df(model_type = model_type,
                                target_var = col)

In [ ]:
eval_df

Which is best so far? 

In [ ]:
best_quantity_rmse = eval_df[eval_df.target_var == 'quantity']['rmse'].min()

In [ ]:
best_quantity_rmse

In [ ]:
eval_df[eval_df.rmse == best_quantity_rmse]

In [ ]:
best_sales_total_rmse = eval_df[eval_df.target_var == 'sales_total']['rmse'].min()

eval_df[eval_df.rmse == best_sales_total_rmse]

As far as baselines are concerned, it looks like our 104 day moving average is a good starting point for comparisons.

# Non-Baseline Models

## Holt-Winters

Two of the models that we will evaluate are based on Holt-Winters, which models on three elements:
- A typical value (average)
- A slope (trend) over time
- And a cyclical repeating pattern (seasonality)

This means that its worth looking at a seasonal decomposition plot of our target, to inspect these components:

**Seasonal Decomposition**

In [ ]:
for col in train.columns:
    sm.tsa.seasonal_decompose(train[col].resample('W').mean()).plot()

It looks like there is both strong seasonality and a notable trend in both targets (`sales_total` and `quantity`). There are two Holt-Winters models that we will attempt
- Holt's Linear Model
- Holt's Seasonal Model

### Holt's Linear Model

Our approach will be similar to many other modeling processes we have performed:
1. Create the object: `Holt()`
2. Fit the object: `.fit()`
3. Make predictions: `.predict()`

The first set of hyperparameters are set when we call `Holt()`: 

- **exponential** = True/False (exponential vs. linear growth, additive vs. multiplicative)
- **damped $\phi$** = True/False
    - with Holt, forecasts tend to increase or decrease indefinitely into the future.  To avoid absurd long term predictions, use the Damped method (True) which sets a damping parameter between 0< ϕ <1. 

A second set of hyperparameters are set when we call `.fit()`: 

- **smoothing_level ($\alpha$)**: value between (0,1)
    - Closer to 0, the level doesn't change with each new observation
    - Closer to 1, the level reacts strongly with each new observation
- **smoothing_slope ($\beta$)**: value between (0,1)
    - Closer to 0, trend is not changing over time
    - Closer to 1, trend is changing significantly over time
- **optimized**: use the auto-optimization that allow statsmodels to automatically find an optimized value for us. 

In [ ]:
col = 'sales_total' 
# create our Holt Object
model = Holt(train[col], exponential=False, damped=True)

In [ ]:
# fit the Holt object
model = model.fit(optimized=True)

In [ ]:
yhat_sales_total = model.predict(start = validate.index[0],
                              end = validate.index[-1])

In [ ]:
validate.shape

In [ ]:
yhat_sales_total

We've successfully created an array of predictions on the validate set for `sales_total`. Lets run a loop that will create a dataframe of predictions for every column in our training set:

In [ ]:
# doing this in a loop for each column
for col in train.columns:
    model = Holt(train[col], exponential=False, damped=True)
    model = model.fit(optimized=True)
    yhat_values = model.predict(start = validate.index[0],
                              end = validate.index[-1])
    yhat_df[col] = round(yhat_values, 2)

In [ ]:
yhat_df.head()

**Plot Actual vs. Predicted Values**

In [ ]:
for col in train.columns:
    plot_and_eval(target_var = col)

Just from visual inspection, Holt's Linear Trend doesn't seem to be that much better than our other baselines. It fails to adequately predict the strong seasonality present in our data. This is not surprising. Holt's Linear Trend tends to work better with data with a strong trend and limited variance. 

**Evaluate**

In [ ]:
for col in train.columns:
    eval_df = append_eval_df(model_type = 'holts_optimized', 
                            target_var = col)

In [ ]:
eval_df.sort_values(by='rmse')

### Holt's Seasonal Trend

Holt's Seasonal Trend is started by using `ExponentialSmoothing()`

The process is similar to our previous model:
1. Create the object: `ExponentialSmoothing()`
2. Fit the object: `.fit()`
3. Make predictions: `.forecast()`

This function has several hyperparameters:
- **seasonal_periods**: The number of periods representing one cycle of seasonality. This is why performing a decomposition plot can be valuable, as this number needs to be entered manually.
- **trend**: Whether the overall trend is additive (`trend='add'`) or multiplicative (`trend='mul'`)
- **seasonal**: Whether the seasonality is additive (`seasonal='add'`) or multiplicative (`seasonal='mul'`)
- **damped**: If we want the trend to reduce over the length of the forecast to avoid absurd long term predictions, we can set `damped=True`

Given our smaller dataset, rather than choosing any one combination of hyperparameters, we can create multiple models to test different combinations:

___
<center>Quantity</center>

|model_name|seasonal_periods|trend|seasonal|damped|
|---|---|---|---|---|
|hst_quantity_fit1|365|add|add|False|
|hst_quantity_fit2|365|add|mul|False|
|hst_quantity_fit3|365|add|add|True|
|hst_quantity_fit4|365|add|mul|True|

___
<center>Sales Total</center>

|model_name|seasonal_periods|trend|seasonal|damped|
|---|---|---|---|---|
|hst_sales_fit1|365|add|add|False|
|hst_sales_fit2|365|add|mul|False|
|hst_sales_fit3|365|add|add|True|
|hst_sales_fit4|365|add|mul|True|

In [ ]:
# Models for quantity
hst_quantity_fit1 = ExponentialSmoothing(train.quantity, seasonal_periods=365, trend='add', seasonal='add').fit()
hst_quantity_fit2 = ExponentialSmoothing(train.quantity, seasonal_periods=365, trend='add', seasonal='mul').fit()
hst_quantity_fit3 = ExponentialSmoothing(train.quantity, seasonal_periods=365, trend='add', seasonal='add', damped=True).fit()
hst_quantity_fit4 = ExponentialSmoothing(train.quantity, seasonal_periods=365, trend='add', seasonal='mul', damped=True).fit()

# Models for sales
hst_sales_fit1 = ExponentialSmoothing(train.sales_total, seasonal_periods=365, trend='add', seasonal='add').fit()
hst_sales_fit2 = ExponentialSmoothing(train.sales_total, seasonal_periods=365, trend='add', seasonal='mul').fit()
hst_sales_fit3 = ExponentialSmoothing(train.sales_total, seasonal_periods=365, trend='add', seasonal='add', damped=True).fit()
hst_sales_fit4 = ExponentialSmoothing(train.sales_total, seasonal_periods=365, trend='add', seasonal='mul', damped=True).fit()

If curious, we can visualize the hyperparameters and coefficients set for each of our models by using `.params`

In [ ]:
hst_quantity_fit1.params

The model will also contain a SSE attribute that we can use to compare performance. We can derive RMSE from SSE, but for now, we can just use SSE to look at the relative performance of our Holt's Seasonal Trend models. 

In [ ]:
results_quantity=pd.DataFrame({'model':['hst_quantity_fit1', 'hst_quantity_fit2', 'hst_quantity_fit3', 'hst_quantity_fit4'],
                              'SSE':[hst_quantity_fit1.sse, hst_quantity_fit2.sse, hst_quantity_fit3.sse, hst_quantity_fit4.sse]})
results_quantity

In [ ]:
results_quantity.sort_values(by='SSE')

For quantity, the 1st version of our Holt's Seasonal Trend model (`trend='add', seasonal='add', damped=False`) is the best performing of the group.

In [ ]:
results_sales=pd.DataFrame({'model':['hst_sales_fit1', 'hst_sales_fit2', 'hst_sales_fit3', 'hst_sales_fit4'],
                              'SSE':[hst_sales_fit1.sse, hst_sales_fit2.sse, hst_sales_fit3.sse, hst_sales_fit4.sse]})
results_sales

In [ ]:
results_sales.sort_values(by='SSE')

Similarly for `sales_total`, the 1st version of our Holt's Seasonal Trend model (`trend='add', seasonal='add', damped=False`) is the best performing of the group.

### Make Predictions

The `.forecast()` method for Holt's Seasonal models requires the number of periods the model is going to provide a prediction for **after** the end of the training data. 

In other words `.forecast(2)` would provide predictions for `2015-07-03` and `2015-07-04`. Therefore, we can get predictions for every day in our validate index by passing the number of rows in validate.

In [ ]:
yhat_df = pd.DataFrame({'sales_total': hst_sales_fit1.forecast(validate.shape[0]),
                           'quantity': hst_quantity_fit1.forecast(validate.shape[0])},
                          index=validate.index)
yhat_df

In [ ]:
for col in train.columns:
    plot_and_eval(col)

In [ ]:
eval_df

In [ ]:
for col in train.columns:
    eval_df = append_eval_df(model_type = 'holts_seasonal_add_add', 
                            target_var = col)

In [ ]:
eval_df.sort_values(by='rmse')

Our best implementation of Holt's Seasonal Trend is significantly outperforming all other models made thus far.

## Previous Cycle

Take all the 2016 data points, compute the daily delta, year-over-year, average that delta over all the days, and adding that average to the previous year's value on a day will give you the forecast for that day. 

If a primary cycle is weekly, then you may want to do this on a week-over-week cadence. 

In the below example:  
1. Compute the 365 average year over year differences from 2013 through 2015
2. Add that average delta to the values during 2015. 
3. Set the index in your yhat dataframe to represent the dates those predictions are make for. 

Let's get started....

**Re-split data**

In [ ]:
train = df_resampled[:'2015']
validate = df_resampled['2016']
test = df_resampled['2017']

print(train.shape)
print(validate.shape)
print(test.shape)

train.head()
train.tail()

**Make Predictions**

In [ ]:
train.diff(365)

In [ ]:
# finding the year-over-year difference for each day from 2013 to 2015
# taking the mean, and then adding that value to the daily 2015 values. 

# find yoy diff. from 2013-2014 and 2014-2015, take the mean, and add to each value in 2015. 
yhat_df = train['2015'] + train.diff(365).mean()
yhat_df

In [ ]:
train.diff(365).mean()

Back to predictions

In [ ]:
train.loc['2015'].head()

In [ ]:
yhat_df.head()

In [ ]:
# let's peek into the prediction we will make for 1/1/2016
# by comparing the predicted value 
# (2015 value + year-over-year average difference)
# to the actual 1/1/2016 value
pd.concat([yhat_df.head(1), validate.head(1)])

In [ ]:
# set yhat_df to index of validate
yhat_df.index = validate.index

In [ ]:
yhat_df.shape

In [ ]:
validate.shape # A leap year!

In [ ]:
validate = validate[validate.index != '2016-02-29']

In [ ]:
yhat_df

In [ ]:
# set yhat_df to index of validate
# yhat_df: 2015 values + the mean year over year difference for the entire training dataset
yhat_df.index = validate.index

In [ ]:
yhat_df.head()

In [ ]:
yhat_df.describe()

In [ ]:
yhat_df.shape

**Plot and Evaluate**

In [ ]:
for col in train.columns:
    plot_and_eval(target_var = col)
    eval_df = append_eval_df(model_type = "previous_year", 
                            target_var = col)

## Conclusion

Which model did the best? 

In [ ]:
eval_df.sort_values(by='rmse')

In [ ]:
sales_total_min_rmse = eval_df.groupby('target_var')['rmse'].min()[0]

quantity_min_rmse = eval_df.groupby('target_var')['rmse'].min()[1]

# find which model that is
eval_df[((eval_df.rmse == sales_total_min_rmse) | 
         (eval_df.rmse == quantity_min_rmse))]

# Performance on Test

Now that we have identified our one best model for each target variable, we can evaluate its peformance on our test data. As a reminder, `.forecast()` allows us to make predictions, but the method always starts after the end of the training data. To get to the test data, we will need to increase the number of periods being predicted to be equal to the combined periods of validate and test. 

We altered our train-validate-test split to perform the previous cycle approach. Let's reset to the original train-validate-test split.

In [ ]:
train = df_resampled[:train_size]
validate = df_resampled[train_size:validate_end_index]
test = df_resampled[validate_end_index:]

In [ ]:
train.shape, validate.shape, test.shape

In [ ]:
yhat_df = pd.DataFrame({'sales_total': hst_sales_fit1.forecast(validate.shape[0] + test.shape[0]),
                           'quantity': hst_quantity_fit1.forecast(validate.shape[0] + test.shape[0])})
yhat_df

In [ ]:
validate.head(1)

In [ ]:
test.head(1)

The original test set started on 2017-01-01, so we can slice out that portion from our `yhat_df`:

In [ ]:
yhat_df = yhat_df['2017-01-01':]

In [ ]:
def final_plot(target_var):
    plt.figure(figsize=(12,4))
    plt.plot(train[target_var], color='#377eb8', label='train')
    plt.plot(validate[target_var], color='#ff7f00', label='validate')
    plt.plot(test[target_var], color='#4daf4a',label='test')
    plt.plot(yhat_df[target_var], color='#a65628', label='yhat')
    plt.legend()
    plt.title(target_var)
    plt.show()

In [ ]:
rmse_sales_total = sqrt(mean_squared_error(test['sales_total'], 
                                       yhat_df['sales_total']))

rmse_quantity = sqrt(mean_squared_error(test['quantity'], 
                                       yhat_df['quantity']))

In [ ]:
print('FINAL PERFORMANCE OF MODEL ON TEST DATA')
print('rmse-sales total: ', rmse_sales_total)
print('rmse-quantity: ', rmse_quantity)
for col in train.columns:
    final_plot(col)

Our RMSE of our final model did get noticably worse on the test data. Its possible that while the performance of the model starts high, it degrades the further out it projects. Let's take a look at what a projection into 2018 would look like.

## Forcasting Into Future

Predicting 2018 simply requires us to extend the value passed to `.forecast()` by an additional 365 periods and then slicing out what we want:

In [ ]:
forecast = pd.DataFrame({'sales_total': hst_sales_fit1.forecast(validate.shape[0] + test.shape[0] + 365),
                           'quantity': hst_quantity_fit1.forecast(validate.shape[0] + test.shape[0] + 365)})
forecast = forecast['2018':]
forecast

In [ ]:
def final_plot(target_var):
    plt.figure(figsize=(12,4))
    plt.plot(train[target_var], color='#377eb8', label='Train')
    plt.plot(validate[target_var], color='#ff7f00', label='Validate')
    plt.plot(test[target_var], color='#4daf4a', label='Test')
    plt.plot(yhat_df[target_var], color='#a65628', label='yhat')
    plt.plot(forecast[target_var], color='#984ea3', label='Forecast')
    plt.title(target_var)
    plt.legend()
    plt.show()

In [ ]:
for col in train.columns:
    final_plot(col)

This data set was made of synthetic data with a clear and observable pattern. It allows us to observe the risk of long term performance degredation. While `Holt's Seasonal Trend` outperformed `Previous Cycle` on validate, `Previous Cycle` would have probably been the best model to use. `Holt's Seasonal Trend` is failing to demonstrate the higher maximum value in each subsequent cycle. 

>**However, I can be confident in this claim because I have atypical knowledge that this data is extremely predictable. Real data is rarely as reliable, and conclusions are almost never as clear as the example we have shown here. Being overconfident about predicting the future is a sure way to get noticed, but maybe not in the way that you would like.** 

Forecasting is an art as much as it is a science, as the environment that created the values of our historical data may be very different from the environment that creates future values. 

>**"It's tough to make predictions, especially about the future." - Yogi Berra**

## Exercises

The end result of this exercise should be a Jupyter notebook named `model`.

Using [saas.csv](https://ds.codeup.com/saas.csv) or log data from API usage

1. Split data (train/test) and resample by any period, except daily, and aggregate using the sum. 
2. Forecast, plot and evaluate using each at least 4 of the methods we discussed:
    - Last Observed Value
    - Simple Average
    - Moving Average
    - Holt's Linear Trend 
    - Holt's Seasonal Trend
    - Based on previous year/month/etc., this is up to you.

Bonus: 
1. Using the store item demand data, create a forecast of `sales_total` and `quantity` for 2018 using the `Previous Cycle` approach.  .  
2. Predict 2018 total **monthly** sales for a single store and/or item by creating a model using prophet.
3. Return a dataframe with the month, store_id, y-hat, and the confidence intervals (y-hat lower, y-hat upper).
4. Plot the 2018 monthly sales predictions.